<a href="https://colab.research.google.com/github/22f3001059/MLP/blob/main/MLP_NPPE_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv('T124OPPE2_Preprocessing_V1.csv')

Saving T124OPPE2_Preprocessing_V1.csv to T124OPPE2_Preprocessing_V1.csv


What is the total number of missing or unknown values in the column Gender?

In [3]:
df.isnull().sum()

,0
Gender,0
Age,0
HasTension,0
AnyHeartDisease,0
NeverMarried,0
Occupation,0
LivesIn,0
GlucoseLevel,0
BMI,149
SmokingStatus,0


In [4]:
df['Gender'].value_counts()

,count
Gender,
Female,2366
Male,1627
Unknown,7


In [5]:
#nothing was missing lol, meaning koi aur value hogi
print(df['Gender'].isin(['Unknown']).sum())

7


What is the total number of missing or unknown values in the column Age?

In [6]:
print(df[(df['Age'] <= 0)]['Age'].count())

8


What is the total number of missing or unknown values in the column GlucoseLevel?

In [7]:
print(df[(df['GlucoseLevel'] <= 0)]['GlucoseLevel'].count())

9



What is the total number of missing or unknown values in the column LivesIn?

In [9]:
df['LivesIn'].value_counts()

,count
LivesIn,
City,2030
Village,1965
Unknown,5


What is the total number of missing or unknown values in the column BMI?

In [11]:
print(df['BMI'].isna().sum())

149


What is the total number of missing or unknown values in the column SmokingStatus?

In [12]:
df['SmokingStatus'].value_counts()

,count
SmokingStatus,
never smoked,1502
Unknown,1204
formerly smoked,697
smokes,597


What is the mean value of the BMI in the dataset? Ignore the missing values if any

In [14]:
print(df[(df['BMI']>0)]['BMI'].mean())

28.857958971695663


How many people live in city, smoked at least once in life and had a heartattack? Ignore records/rows with any missing values.

In [22]:
df[(df['LivesIn']=='City')&(df['HeartAttack']=='Yes')&((df['SmokingStatus'] == 'smokes')|(df['SmokingStatus'] == 'formerly smoked'))].shape[0]

52

Which of the following categories have highest frequency? Ignore rows with missing values.

* female patients without tension, without any heart disease and never married

* female patients without tension, without any heart disease and either currently married or married before

* male patients without tension, without any heart disease and never married

* male patients with tension, with a heart disease and never married

In [25]:
option1 = df[(df['Gender']=='Female')&(df['HasTension']=='No')&(df['AnyHeartDisease']=='No')&(df['NeverMarried']=='Yes')].shape[0]
option2 = df[(df['Gender']=='Female')&(df['HasTension']=='No')&(df['AnyHeartDisease']=='No')&(df['NeverMarried']=='No')].shape[0]
option3 = df[(df['Gender']=='Male')&(df['HasTension']=='No')&(df['AnyHeartDisease']=='No')&(df['NeverMarried']=='Yes')].shape[0]
option4 = df[(df['Gender']=='Male')&(df['HasTension']=='Yes')&(df['AnyHeartDisease']=='Yes')&(df['NeverMarried']=='Yes')].shape[0]
print(f'{option1},{option2},{option3},{option4}')



1335,754,795,24


In [27]:
print(df.select_dtypes(include='object').columns)

Index(['Gender', 'HasTension', 'AnyHeartDisease', 'NeverMarried', 'Occupation',
       'LivesIn', 'SmokingStatus', 'HeartAttack'],
      dtype='object')


In [28]:
df['HeartAttack'].value_counts()

,count
HeartAttack,
No,3806
Yes,194


Divide the data into training and test sets

Keep 30% of the data as test set.
Use random_state as 0
HeartAttack is the target, rest of the columns are the features.
 For the label/target vector, replace "Yes" with 1 and "No" with 0.
 Divide the dataset into training and test sets keeping target(y) in stratified manner.

Hint: look for the documentation of the usual function that divides the data into training and test datasets.

Prepare a data preprocessing pipeline to process features in following order:

Gender: Impute with most frequent then ordinally encode.

Age: Impute with mean then standard scale.

HasTension: Ordinally encode.

AnyHeartDisease:Ordinally encode.

NeverMarried:Ordinally encode.

Occupation: One hot encode.

LivesIn: Impute with most frequent then ordinally encode.

GlucoseLevel: Impute with mean, then min-max scaling.

BMI: Impute with mean, then standard scale.

SmokingStatus: Impute with most frequent, then one hot encode

Hint: After transformation, your feature matrix must have columns in following order:

	0. Gender
	1. Age
	2. HasTension
	3. AnyHeartDisease
	4. NeverMarried
	5. Occupation_Govt_job
	6. Occupation_Never_worked
	7. Occupation_Private
	8. Occupation_Self-employed
	9. Occupation_children
	10. LivesIn
	11. GlucoseLevel
	12. BMI
	13. SmokingStatus_formerly smoked
	14. SmokingStatus_never smoked
	15. SmokingStatus_smokes

In [29]:
from re import X
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(df.drop('HeartAttack', axis=1), df['HeartAttack'], test_size=0.3, random_state=0, stratify=df['HeartAttack'])

preprocessor = ColumnTransformer([
    ('gender', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoder', OrdinalEncoder())
    ]), ['Gender']),
    ('age', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('standard_scaler', StandardScaler())
    ]), ['Age']),
    ('has_tension', Pipeline([
        ('ordinal_encoder', OrdinalEncoder())
        ]), ['HasTension']),
    ('any_heart_disease', Pipeline([
        ('ordinal_encoder', OrdinalEncoder())
        ]), ['AnyHeartDisease']),
    ('never_married', Pipeline([
        ('ordinal_encoder', OrdinalEncoder())
        ]), ['NeverMarried']),
    ('occupation', Pipeline([
        ('one_hot_encoder', OneHotEncoder())
        ]), ['Occupation']),
    ('lives_in', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoder', OrdinalEncoder())
    ]), ['LivesIn']),
    ('glucose_level', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('min_max_scaler', MinMaxScaler())
    ]), ['GlucoseLevel']),
    ('bmi', Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('standard_scaler', StandardScaler())
    ]), ['BMI']),
    ('smoking_status', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('one_hot_encoder', OneHotEncoder())
    ]), ['SmokingStatus'])
])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)



In [30]:
X_train.shape

(2800, 17)